# Descriptor testing

In [285]:
import numbers


class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        raise AttributeError('Deleting attribute not allowed')


class Typed(Descriptor):
    expected_type = type(None)

    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('Expected ' + str(self.expected_type))
        super().__set__(instance, value)


class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)
    
    
class Real(Typed):
    expected_type = numbers.Real


class String(Typed):
    expected_type = str
    

class UnsignedReal(Real, Unsigned):
    pass


class Vector3D(Typed):
    expected_type = tuple
    
    def __set__(self, instance, value):
        if len(value) != 3:
            raise ValueError('Expected 3D vector.')
        super().__set__(instance, value)


class RealVector3D(Vector3D):
    def __set__(self, instance, value):
        if not all([isinstance(i, numbers.Real) for i in value]):
            raise ValueError('Expected Real vector components.')
        super().__set__(instance, value)

In [286]:
def typesystem(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))  # not very clear!
        
        return cls
    return decorate

In [287]:
@typesystem(K=Real, u=RealVector3D)
class Anisotropy:
    def __init__(self, K, u):
        self.K = K
        self.u = u

In [288]:
a1 = Anisotropy(1e6, (0, 1, 0))

In [289]:
a1.K

1000000.0

In [290]:
a1.K = (1,2)

TypeError: Expected <class 'numbers.Real'>

In [291]:
a1.K

1000000.0

In [292]:
a1.u

(0, 1, 0)

In [293]:
a2 = Anisotropy(6.6e9, (0, 0, 1))

In [294]:
a1.K

1000000.0

In [295]:
a2.K

6600000000.0

In [296]:
a1.u

(0, 1, 0)

In [297]:
a1.u = (5, 6)

ValueError: Expected 3D vector.

In [282]:
a1.u = (5, 6, 7)

In [283]:
a1.u

(5, 6, 7)

In [284]:
a2.u

(0, 0, 1)